In [2]:

from gensim.models import KeyedVectors
import os, sys, json, numpy as np

In [5]:
with open('/itf-fi-ml/shared/users/ziyuzh/svm/data/bioconcept/concept_fast.json') as json_file:  
    concept_vectors = json.load(json_file)
print('load', len(concept_vectors), 'concepts')

load 402712 concepts


In [40]:
concept_vectors

{'Chemical_MESH_C532751': [0.28876999020576477,
  -0.03438200056552887,
  -0.03456899896264076,
  0.12304999679327011,
  0.36360999941825867,
  0.12020999938249588,
  -0.014631999656558037,
  0.30842000246047974,
  -0.10683000087738037,
  -0.31477999687194824,
  0.18859000504016876,
  -0.30204999446868896,
  -0.4241800010204315,
  0.24244000017642975,
  -0.4611699879169464,
  -1.2271000146865845,
  -0.5400500297546387,
  -0.0855029970407486,
  0.17364999651908875,
  -0.09370599687099457,
  -0.5351600050926208,
  -0.8492699861526489,
  -0.14696000516414642,
  0.2777999937534332,
  0.39893999695777893,
  0.46110999584198,
  0.7101799845695496,
  -0.5430099964141846,
  0.44679000973701477,
  0.29183000326156616,
  -0.5434799790382385,
  0.08865000307559967,
  -0.20514999330043793,
  0.22944000363349915,
  0.10570000112056732,
  0.3684900104999542,
  0.45500999689102173,
  0.17678000032901764,
  0.06624799966812134,
  -0.5969700217247009,
  -0.17315000295639038,
  0.8524799942970276,
  -0.

In [14]:
gene_ids = [key for key, value in concept_vectors.items() if key.startswith('Gene')]

In [36]:
gene_unnest = dict()
all_genes = []
for i in gene_ids:
    all_genes.extend(i.split('_')[1:])
    for j in i.split('_')[1:]:
        gene_unnest[j] = i
    

In [21]:
import mygene

mg = mygene.MyGeneInfo()
map_df = mg.getgenes(all_genes, fields='name,symbol,entrezgene,taxid',as_dataframe=True)

Input sequence provided is already in string format. No operation performed


In [23]:
len(map_df)

173165

In [24]:
map_id = map_df[['_id','symbol']]
map_id = map_id.drop_duplicates(subset='symbol')
len(map_id)

79941

In [25]:
import pandas as pd

In [26]:
import os
local_stringdb = '/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb/2023'

ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.info.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
ppidf['preferred_name'] = ppidf['preferred_name'].str.upper()
stringId2name = ppidf.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = ppidf.set_index('preferred_name')['#string_protein_id'].to_dict()
ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.aliases.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
ppidf['alias'] = ppidf['alias'].str.upper()
aliases2stringId = ppidf.set_index('alias')['#string_protein_id'].to_dict()

def string_convert(gene):
    if gene in name2stringId.keys():
        return name2stringId[gene]
    elif gene in aliases2stringId.keys():
        return aliases2stringId[gene]
    else:
        return None

In [27]:
entrz2string = dict()
for gene in map_df['symbol']:
    entrz2string[gene] = string_convert(gene)

In [28]:
map_id['string_id'] = map_id['symbol'].map(entrz2string)
map_id = map_id.reset_index()

In [29]:
map_id = map_id.drop_duplicates(subset='string_id')
len(map_id)

16973

In [32]:
map_id.dropna(subset='string_id', inplace=True)

In [34]:
map_id.head(3)

,query,_id,symbol,string_id
3,10201221,10201221,CYTB,9606.ENSP00000354554
4,80344,80344,DCAF11,9606.ENSP00000415556
5,823753,823753,HRS1,9606.ENSP00000341963


In [49]:
map_id['row_key'] = map_id["query"].map(gene_unnest)

In [52]:
gene_ids_features = {key:concept_vectors[key] for key in map_id['row_key']}

In [56]:
len(gene_ids_features['Gene_100008580'])

100

In [57]:
gene_ids_features_df = pd.DataFrame.from_dict(gene_ids_features, orient='index')

# To rename the columns if needed
gene_ids_features_df.columns = [f'feature_{i}' for i in range(100)]

In [59]:
map_id.head(3)

,query,_id,symbol,string_id,row_key
3,10201221,10201221,CYTB,9606.ENSP00000354554,Gene_10201221
4,80344,80344,DCAF11,9606.ENSP00000415556,Gene_80344_1642
5,823753,823753,HRS1,9606.ENSP00000341963,Gene_823753


In [60]:
gene_ids_features_df['string_id'] = gene_ids_features_df.index.map(map_id.set_index('row_key')['string_id'].to_dict())

In [62]:
gene_ids_features_df.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/bioconcept/bioconcept_full.csv',index=False)